In [29]:
print("X.")

X.


In [2]:
%%time

import json
reviews = []
with open("yelp_academic_dataset_review.json") as f:
    index = 0
    for line in f:
        if index % 1000000 == 0:
            print(index)
        index += 1
        reviews.append(json.loads(line))

0
1000000
2000000
3000000
4000000
CPU times: user 46.4 s, sys: 6.06 s, total: 52.4 s
Wall time: 25min 42s


In [13]:
print(reviews[0])

{'user_id': 'KpkOkG6RIf4Ra25Lhhxf1A', 'stars': 5, 'date': '2011-10-10', 'useful': 0, 'type': 'review', 'text': "If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.", 'cool': 0, 'business_id': '2aFiy99vNLklCx3T_tGS9A', 'funny': 0, 'review_id': 'NxL8SIC5yqOdnlXCg18IBg'}


In [14]:
from collections import Counter
prolific_reviewers = Counter([review['user_id'] for review in reviews]).most_common(30)

In [15]:
keep_ids = {pr[0] : 0 for pr in prolific_reviewers}

In [16]:
by_author = {} # author : "review 1\n review 2\n review 3"
for review in reviews:
    uid = review['user_id']
    if uid in keep_ids:
        uid = review['user_id']
        if uid in by_author:
            by_author[uid] += "\n{}".format(review['text'])
        else:
            by_author[uid] = "{}".format(review['text'])


In [17]:
len(by_author)

30

In [18]:
# check that we have at least 200000 characters for each author
sorted([(len(by_author[key]), key) for key in by_author])[:5]

[(276136, 'ffPY_bHX8vLebHu8LBEqfg'),
 (278427, 'PeLGa5vUR8_mcsn-fn42Jg'),
 (351311, 'cMEtAiW60I5wE_vLfTxoJQ'),
 (370129, 'iDlkZO2iILS8Jwfdy7DP9A'),
 (461333, 'dt9IHwfuZs9D9LOH7gjNew')]

In [19]:
def get_chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

In [20]:
train_texts = []  # the first 100 000 chars from each author
train_labels = [] # each author
test_texts = []   # 100 texts of 1000 characters each (second 100 000 chars of each author)
test_labels = []  # each author * 100

author_int = {author: i for i,author in enumerate(by_author)}
int_author = {author_int[author]: author for author in author_int}

for author in by_author:
    train_text = by_author[author][:5000]
    train_label = author_int[author]
    train_texts.append(train_text)
    train_labels.append(train_label)
    
    short_texts = get_chunks(by_author[author][5000:10000], 1000)
    for text in short_texts:
        test_texts.append(text)
        test_labels.append(author_int[author])

In [21]:
print(len(train_texts), len(test_texts))

30 150


In [24]:
max([len(x) for x in test_texts])

1000

In [22]:
len(train_texts[0])

5000

In [31]:
'''This script loads pre-trained word embeddings (GloVe embeddings)
into a frozen Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classication of newsgroup messages into 20 different categories).

GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)

20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html
'''

from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model


BASE_DIR = ''
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train_texts)
tr_sequences = tokenizer.texts_to_sequences(train_texts)
te_sequences = tokenizer.texts_to_sequences(test_texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

tr_data = pad_sequences(tr_sequences, maxlen=5000)
te_data = pad_sequences(te_sequences, maxlen=5000)

tr_labels = to_categorical(np.asarray(train_labels))
te_labels = to_categorical(np.asarray(test_labels))
print('Shape of data tensor:', tr_data.shape)
print('Shape of label tensor:', tr_labels.shape)

Indexing word vectors.
Found 400000 word vectors.
Found 4512 unique tokens.
Shape of data tensor: (30, 5000)
Shape of label tensor: (30, 30)


In [40]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
print(num_words)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        if i>= MAX_NB_WORDS:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    except Exception as e:
        print(i)
        print(e)

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(5000,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(30, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(tr_data, tr_labels,
          batch_size=128,
          epochs=10)

Preparing embedding matrix.
4512
4512
index 4512 is out of bounds for axis 0 with size 4512
Training model.
Epoch 1/10


InvalidArgumentError: Matrix size-incompatible: In[0]: [30,640], In[1]: [128,128]
	 [[Node: dense_8/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](flatten_5/Reshape, dense_8/kernel/read)]]
	 [[Node: Mean_15/_97 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_25_Mean_15", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'dense_8/MatMul', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.4/dist-packages/traitlets/config/application.py", line 658, in laun

In [41]:
# vectorization - chars to ints
import string
import random
import sys

import numpy as np

from keras.models import load_model

def sample(preds, temperature=1.0):
    """Sample predictions from a probability array"""
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-6) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate(model, diversity=0.5, text=""):
    """Generate text from a model"""
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(5000):
        x = np.zeros((1, maxlen), dtype=np.int)
        for t, char in enumerate(sentence):
            try:
                x[0, t] = char_indices[char]
            except:
                print(sentence)
        preds = model.predict(x, verbose=0)[0][0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    return

def vectorize(text):
    """Convert text into character sequences"""
    step = 3
    sentences = []
    next_chars = []
    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    X = np.zeros((len(sentences), maxlen), dtype=np.int)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t] = char_indices[char]
        y[i, char_indices[next_chars[i]]] = 1
    return X, y

def clean_text(text, charset):
    text = " ".join(text.split())  # all white space is one space
    text = "".join([x for x in text if x in charset])  # remove characters that we don't care about
    return text

def get_model(modelfile, freeze=False):
    model = load_model(modelfile)
    if freeze:
        for layer in model.layers[:6]:
            layer.trainable = False
    return model

chars = " " + string.ascii_letters + string.punctuation  # sorted to keep indices consistent
charset = set(chars)  # for lookup
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 100  # must match length which generated model - the sequence length

# load a pretrained language model
modelfile = "charlm2/model_middlemarch_cnn.hdf5"

In [44]:
# test train an author specific model
# test_author_text = clean_text(train_texts[0], charset)
# test_author_model = get_model(modelfile, freeze=False)
X, y = vectorize(test_author_text)

NameError: name 'test_author_text' is not defined

In [42]:
test_author_model = get_model(modelfile, freeze=True)
test_author_model.compile(loss="categorical_crossentropy", optimizer="adam", loss_weights=[1., 0.2])
test_author_model.fit(X, [y, y], epochs=5, batch_size=128, validation_split=0.1)

OSError: Unable to open file (Unable to open file: name = 'charlm2/model_middlemarch_cnn.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [ ]:
test_author_model = get_model(modelfile, freeze=True)
test_author_model.fit(X, [y, y], epochs=10, batch_size=128, validation_split=0.1)

In [ ]:
from keras.models import model_from_json
fresh_model = model_from_json(test_author_model.to_json())
fresh_model.compile(loss="categorical_crossentropy", optimizer="adam", loss_weights=[1.0, 0.2])

In [ ]:
test_author_model.fit(X, [y, y], epochs=5, batch_size=128, validation_split=0.1)

In [ ]:
test_author_model.optimizer

In [ ]:
from statistics import mean

print(mean(scores[:50]))
print(mean(scores[50:]))

In [ ]:
len(scores)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
plt.boxplot(get_chunks(scores, 50)[:20]);

In [ ]:
scores[:10]

In [ ]:
test_labels[:50]

In [ ]:
generate(test_author_model, diversity=0.7, text="this is some test text does it really matter what it says "*30)

In [80]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Input, Embedding, Conv1D, MaxPooling1D, BatchNormalization, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

def get_gru_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(charset), output_dim=100))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(GRU(256))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [81]:
%%time
X, y = vectorize(clean_text(' '.join(train_texts), charset))
general_model = get_gru_model()
general_model.fit(X, y, epochs=10, batch_size=128, validation_split=0.1)

Train on 44456 samples, validate on 4940 samples
Epoch 1/10
44456/44456 [==============================] - 50s - loss: 2.7679 - val_loss: 2.7833
Epoch 2/10
44456/44456 [==============================] - 48s - loss: 2.1750 - val_loss: 1.9799
Epoch 3/10
44456/44456 [==============================] - 48s - loss: 2.0096 - val_loss: 1.8864
Epoch 4/10
44456/44456 [==============================] - 48s - loss: 1.8989 - val_loss: 1.8278
Epoch 5/10
44456/44456 [==============================] - 48s - loss: 1.8144 - val_loss: 1.7907
Epoch 6/10
44456/44456 [==============================] - 48s - loss: 1.7366 - val_loss: 1.7575
Epoch 7/10
44456/44456 [==============================] - 48s - loss: 1.6695 - val_loss: 1.7439
Epoch 8/10
44456/44456 [==============================] - 48s - loss: 1.6115 - val_loss: 1.7482
Epoch 9/10
44456/44456 [==============================] - 48s - loss: 1.5490 - val_loss: 1.7392
Epoch 10/10
44456/44456 [==============================] - 48s - loss: 1.4917 - val_los

In [82]:
general_model.save_weights("general_yelp_small_weights.hdf5")

In [83]:
def get_general_model():
    model = get_gru_model()
    model.load_weights("general_yelp_small_weights.hdf5")
    for layer in model.layers[:-1]:
        layer.trainable = False
    return model

In [116]:
%%time
author_models = []  # [(author_model, author_id), (author_model, author_id), ...] - ids are ints
for i, train_text in enumerate(train_texts):
    print("{} / {}".format(i, len(train_texts)))
    ct = clean_text(train_text, charset)
    X, y = vectorize(ct)
    am = get_general_model()
    am.fit(X, y, epochs=2, batch_size=128, verbose=0)
    author_models.append((am, train_labels[i]))

0 / 30
1 / 30
2 / 30
3 / 30
4 / 30
5 / 30
6 / 30
7 / 30
8 / 30
9 / 30
10 / 30
11 / 30
12 / 30
13 / 30
14 / 30
15 / 30
16 / 30
17 / 30
18 / 30
19 / 30
20 / 30
21 / 30
22 / 30
23 / 30
24 / 30
25 / 30
26 / 30
27 / 30
28 / 30
29 / 30
CPU times: user 9min 53s, sys: 26.4 s, total: 10min 19s
Wall time: 8min 52s


In [115]:
# am = get_gru_model()
# X, y = vectorize(ct)
# am.fit(X, y, epochs=40, batch_size=128, verbose=1, validation_split=0.1)

In [102]:
ct = clean_text(test_texts[0], charset)
X, y = vectorize(ct)
am = get_general_model()

print(author_models[0][0].evaluate(X, y))
print(am.evaluate(X, y))

298/298 [==============================] - 0s     
2.15473028157
298/298 [==============================] - 3s     
1.80639350174


In [114]:
am = get_general_model()
Xt, yt = vectorize(clean_text(train_texts[2], charset))
print(am.evaluate(X, y))
am.fit(Xt, yt, epochs=3, batch_size=128, validation_data=(X, y))

288/298 [===========================>..] - ETA: 0s1.80639350174
Train on 1601 samples, validate on 298 samples
Epoch 1/3
1601/1601 [==============================] - 6s - loss: 1.9482 - val_loss: 1.8065
Epoch 2/3
1601/1601 [==============================] - 1s - loss: 1.5206 - val_loss: 1.8550
Epoch 3/3
1601/1601 [==============================] - 1s - loss: 1.3027 - val_loss: 1.8758


In [117]:
print(len(author_models))
print(len(test_texts))
print(len(test_labels))

30
150
150


In [118]:
from statistics import mean
word_counts = [text.count(" ") for text in test_texts]
mean(word_counts)

182.18666666666667

In [47]:
# 182 words is quite short
# Try to join 5 tests texts together
longer_test_texts = get_chunks(test_texts, 5)
longer_test_labels = get_chunks(test_labels, 5)

In [48]:
all([len(set(x)) == 1 for x in longer_test_labels])  # Make sure that all combined labels are the same

True

In [47]:
longer_test_texts = ['\n'.join(chunk) for chunk in longer_test_texts]

In [48]:
longer_test_labels = [chunk[0] for chunk in longer_test_labels]

In [119]:
%%time
from random import shuffle
from datetime import datetime

def get_predictions(author_models, test_texts, test_labels):
    """Evaluate each text for each author_model and append first metric to predictions"""
    indicies = list(range(len(test_texts)))

    test_texts = np.array(test_texts)
    test_labels = np.array(test_labels)

    test_texts = test_texts[indicies]
    test_labels = test_labels[indicies]

    predictions = []
    for i, text in enumerate(test_texts):
        t1 = datetime.now()
        print("{} / {}".format(i, len(test_texts)), end=" ")
        X, y = vectorize(clean_text(text, charset))

        losses = []
        for am in author_models:
            print(".", end="")
            model = am[0]
            label = am[1]
            loss = model.evaluate(X, y, verbose=0)
            losses.append((loss, label))
        print(" {}".format(datetime.now() - t1))
        predictions.append(losses)
    return predictions
    

predictions_long = get_predictions(author_models, test_texts, test_labels)

0 / 150 .............................. 0:02:57.151682
1 / 150 .............................. 0:00:10.570115
2 / 150 .............................. 0:00:10.448650
3 / 150 .............................. 0:00:10.518715
4 / 150 .............................. 0:00:10.530489
5 / 150 .............................. 0:00:10.512609
6 / 150 .............................. 0:00:10.488222
7 / 150 .............................. 0:00:10.522617
8 / 150 .............................. 0:00:10.536951
9 / 150 .............................. 0:00:10.534474
10 / 150 .............................. 0:00:10.488669
11 / 150 .............................. 0:00:10.451334
12 / 150 .............................. 0:00:10.516880
13 / 150 .............................. 0:00:10.497487
14 / 150 .............................. 0:00:10.511437
15 / 150 .............................. 0:00:10.525160
16 / 150 .............................. 0:00:10.510449
17 / 150 .............................. 0:00:10.517755
18 / 150 ...........

148 / 150 .............................. 0:00:10.491207
149 / 150 .............................. 0:00:10.517961
CPU times: user 26min 46s, sys: 1.17 s, total: 26min 47s
Wall time: 28min 58s


In [120]:
author_models[0][0] == author_models[1][0]

False

In [121]:
len(predictions_long)

150

In [122]:
pred_is = []
for pred in predictions_long:
    pred_i = [p[0] for p in pred]
    pred_is.append(pred_i)

In [123]:
pred_labs = [np.argmin(pred) for pred in pred_is]

In [124]:
pred_labs

[8,
 19,
 20,
 8,
 11,
 10,
 0,
 14,
 20,
 29,
 13,
 2,
 22,
 16,
 16,
 28,
 21,
 20,
 3,
 1,
 4,
 3,
 29,
 10,
 28,
 12,
 11,
 28,
 13,
 17,
 23,
 4,
 1,
 15,
 1,
 7,
 10,
 4,
 28,
 27,
 26,
 28,
 22,
 27,
 29,
 9,
 27,
 9,
 3,
 3,
 14,
 25,
 28,
 1,
 29,
 17,
 7,
 22,
 29,
 26,
 12,
 26,
 20,
 12,
 14,
 13,
 13,
 20,
 10,
 13,
 27,
 25,
 14,
 16,
 10,
 7,
 19,
 25,
 11,
 1,
 4,
 6,
 16,
 16,
 10,
 26,
 28,
 29,
 1,
 1,
 17,
 12,
 16,
 22,
 17,
 1,
 28,
 17,
 25,
 26,
 29,
 1,
 24,
 20,
 20,
 20,
 12,
 27,
 28,
 1,
 9,
 13,
 23,
 22,
 20,
 16,
 24,
 3,
 6,
 27,
 24,
 23,
 1,
 4,
 29,
 29,
 20,
 2,
 10,
 14,
 18,
 9,
 26,
 9,
 25,
 3,
 25,
 16,
 17,
 29,
 28,
 28,
 1,
 18,
 28,
 29,
 22,
 28,
 26,
 12]

In [127]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, pred_labs)

0.15333333333333332

In [128]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

svm = LinearSVC()
vec = TfidfVectorizer()

X_train = vec.fit_transform(train_texts)

svm.fit(X_train, train_labels)

X_test = vec.transform(test_texts)
preds = svm.predict(X_test)

print(accuracy_score(preds, pred_labs))

0.153333333333


In [56]:
pred_labs

[2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 28,
 2,
 0,
 2,
 2,
 2,
 2,
 2]

In [ ]:
test_author_model.summary()

In [ ]:
test_author_model.evaluatei

In [ ]:
len(train_texts)

In [ ]:
len(train_labels)

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
word_vec = TfidfVectorizer(min_df=3, ngram_range=(1,2))
char_vec = TfidfVectorizer(min_df=3, ngram_range=(2,5))

fu = FeatureUnion([
    ('word', word_vec),
    ('char', char_vec)
])


X_train = fu.fit_transform(train_texts)

In [ ]:
X_test = fu.transform(test_texts)

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC()

In [ ]:
svm.fit(X_train, train_labels)

In [ ]:
preds = svm.predict(X_test)

In [ ]:
accuracy_score(test_labels, preds)

In [ ]:
X_test_longer = fu.transform(longer_test_texts)

In [ ]:
preds = svm.predict(X_test_longer)

In [ ]:
accuracy_score(longer_test_labels, preds)

In [ ]:
X.shape

In [ ]:
X, y = vectorize(clean_text(train_texts[3], charset))

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
y[0]

In [ ]:
X[0]

In [ ]:
test_author_model.layers[:6]

In [30]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Input, Embedding, Conv1D, MaxPooling1D, BatchNormalization, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# load ascii text and covert to lowercase

# cnn = Dropout(0.2)(embedded)
# cnn = Conv1D(128, 5, activation='relu')(cnn)
# cnn = MaxPooling1D(pool_size=4)(cnn)

model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=13, batch_size=128, validation_split=0.1)

"""
LSTM, BatchNorm
Train on 14929 samples, validate on 1659 samples
Epoch 1/5
14929/14929 [==============================] - 75s - loss: 3.0403 - val_loss: 3.4757
Epoch 2/5
14929/14929 [==============================] - 60s - loss: 2.3156 - val_loss: 2.9687

LSTM
Train on 14929 samples, validate on 1659 samples
Epoch 1/5
14929/14929 [==============================] - 75s - loss: 3.1259 - val_loss: 2.7865
Epoch 2/5
14929/14929 [==============================] - 60s - loss: 2.6150 - val_loss: 2.3894

CNN(5), LSTM  # faster, needs more epochs
Train on 14929 samples, validate on 1659 samples
Epoch 1/5
14929/14929 [==============================] - 42s - loss: 3.1579 - val_loss: 2.9987
Epoch 2/5
14929/14929 [==============================] - 26s - loss: 2.8874 - val_loss: 2.6994
Epoch 3/5
14929/14929 [==============================] - 26s - loss: 2.6220 - val_loss: 2.4879
Epoch 4/5
14929/14929 [==============================] - 26s - loss: 2.4309 - val_loss: 2.3942
Epoch 5/5
14929/14929 [==============================] - 26s - loss: 2.2950 - val_loss: 2.2902

CNN(5), CNN(3), LSTM doesn't drop below 3.0 in 5 epochs


Embedding, BatchNorm, GRU, BatchNorm
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 16s - loss: 2.9240 - val_loss: 3.9516
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.2447 - val_loss: 3.3667
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.0054 - val_loss: 2.8011
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 1.8388 - val_loss: 2.3477
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 1.7122 - val_loss: 2.0196
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.6069 - val_loss: 1.9417
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.5044 - val_loss: 1.9541
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.3987 - val_loss: 1.9512
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.2940 - val_loss: 1.9921
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.1850 - val_loss: 2.0424

Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.2577 - val_loss: 3.8892
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.5444 - val_loss: 3.2364
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.2915 - val_loss: 2.7205
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.1258 - val_loss: 2.3477
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 2.0219 - val_loss: 2.0588
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.9386 - val_loss: 1.9478
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.8762 - val_loss: 1.9152
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.7994 - val_loss: 1.9076
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.7224 - val_loss: 1.8829
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.6783 - val_loss: 1.9007
Epoch 11/20
14725/14725 [==============================] - 15s - loss: 1.6154 - val_loss: 1.8910
Epoch 12/20
14725/14725 [==============================] - 15s - loss: 1.5485 - val_loss: 1.8920
Epoch 13/20
14725/14725 [==============================] - 15s - loss: 1.4844 - val_loss: 1.9198
Epoch 14/20
14725/14725 [==============================] - 15s - loss: 1.4291 - val_loss: 1.9193
Epoch 15/20
14725/14725 [==============================] - 15s - loss: 1.3670 - val_loss: 1.9295
Epoch 16/20
14725/14725 [==============================] - 15s - loss: 1.3028 - val_loss: 1.9752
Epoch 17/20
14725/14725 [==============================] - 15s - loss: 1.2321 - val_loss: 1.9969
Epoch 18/20
14725/14725 [==============================] - 15s - loss: 1.2159 - val_loss: 2.0431
Epoch 19/20
14725/14725 [==============================] - 15s - loss: 1.1391 - val_loss: 2.0709
Epoch 20/20
14725/14725 [==============================] - 15s - loss: 1.1321 - val_loss: 2.1172


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.2577 - val_loss: 3.8892
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.5444 - val_loss: 3.2364
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.2915 - val_loss: 2.7205
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.1258 - val_loss: 2.3477
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 2.0219 - val_loss: 2.0588
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.9386 - val_loss: 1.9478
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.8762 - val_loss: 1.9152
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.7994 - val_loss: 1.9076
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.7224 - val_loss: 1.8829
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.6783 - val_loss: 1.9007
Epoch 11/20
14725/14725 [==============================] - 15s - loss: 1.6154 - val_loss: 1.8910
Epoch 12/20
14725/14725 [==============================] - 15s - loss: 1.5485 - val_loss: 1.8920
Epoch 13/20
14725/14725 [==============================] - 15s - loss: 1.4844 - val_loss: 1.9198
Epoch 14/20
14725/14725 [==============================] - 15s - loss: 1.4291 - val_loss: 1.9193
Epoch 15/20
14725/14725 [==============================] - 15s - loss: 1.3670 - val_loss: 1.9295
Epoch 16/20
14725/14725 [==============================] - 15s - loss: 1.3028 - val_loss: 1.9752
Epoch 17/20
14725/14725 [==============================] - 15s - loss: 1.2321 - val_loss: 1.9969
Epoch 18/20
14725/14725 [==============================] - 15s - loss: 1.2159 - val_loss: 2.0431
Epoch 19/20
14725/14725 [==============================] - 15s - loss: 1.1391 - val_loss: 2.0709
Epoch 20/20
14725/14725 [==============================] - 15s - loss: 1.1321 - val_loss: 2.1172


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(GRU(256))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 16s - loss: 3.1731 - val_loss: 3.5648
Epoch 2/20
14725/14725 [==============================] - 14s - loss: 2.4964 - val_loss: 2.9875
Epoch 3/20
14725/14725 [==============================] - 14s - loss: 2.3446 - val_loss: 2.7695
Epoch 4/20
14725/14725 [==============================] - 14s - loss: 2.2928 - val_loss: 2.6010
Epoch 5/20
14725/14725 [==============================] - 14s - loss: 2.2642 - val_loss: 2.3900
Epoch 6/20
14725/14725 [==============================] - 14s - loss: 2.2373 - val_loss: 2.5023
Epoch 7/20
14725/14725 [==============================] - 14s - loss: 2.2186 - val_loss: 2.3780
Epoch 8/20
14725/14725 [==============================] - 14s - loss: 2.2029 - val_loss: 2.4928
Epoch 9/20
14725/14725 [==============================] - 14s - loss: 2.1852 - val_loss: 2.3480
Epoch 10/20
14725/14725 [==============================] - 14s - loss: 2.1745 - val_loss: 2.4801
Epoch 11/20
14725/14725 [==============================] - 14s - loss: 2.1563 - val_loss: 2.3951
Epoch 12/20
14725/14725 [==============================] - 14s - loss: 2.1391 - val_loss: 2.4133
Epoch 13/20
14725/14725 [==============================] - 14s - loss: 2.1192 - val_loss: 2.5896
Epoch 14/20
14725/14725 [==============================] - 14s - loss: 2.1020 - val_loss: 2.2692
Epoch 15/20
14725/14725 [==============================] - 14s - loss: 2.0770 - val_loss: 2.2179
Epoch 16/20
14725/14725 [==============================] - 14s - loss: 2.0643 - val_loss: 2.2822
Epoch 17/20
 6784/14725 [============>.................] - ETA: 7s - loss: 2.0302


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.1528 - val_loss: 3.9042
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.4658 - val_loss: 3.2093
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.2001 - val_loss: 2.6764
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.0358 - val_loss: 2.2919
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 1.9497 - val_loss: 2.0060
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.8588 - val_loss: 1.9313
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.7883 - val_loss: 1.9153
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.7034 - val_loss: 1.9145
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.6382 - val_loss: 1.8979
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.5827 - val_loss: 1.8864
Epoch 11/20
14725/14725 [==============================] - 15s - loss: 1.5093 - val_loss: 1.8967
Epoch 12/20
14725/14725 [==============================] - 15s - loss: 1.4472 - val_loss: 1.9040
Epoch 13/20
14725/14725 [==============================] - 15s - loss: 1.3809 - val_loss: 1.9227
Epoch 14/20
14725/14725 [==============================] - 15s - loss: 1.3225 - val_loss: 1.9469
Epoch 15/20
14725/14725 [==============================] - 15s - loss: 1.2516 - val_loss: 1.9862
Epoch 16/20
14725/14725 [==============================] - 15s - loss: 1.2094 - val_loss: 1.9963
Epoch 17/20
14725/14725 [==============================] - 15s - loss: 1.1658 - val_loss: 2.0331
Epoch 18/20
14725/14725 [==============================] - 15s - loss: 1.0851 - val_loss: 2.0452
Epoch 19/20
14725/14725 [==============================] - 15s - loss: 1.0394 - val_loss: 2.0810
Epoch 20/20
14725/14725 [==============================] - 15s - loss: 0.9903 - val_loss: 2.1283


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.2991 - val_loss: 3.8902
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.5672 - val_loss: 3.1627
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.2731 - val_loss: 2.6340
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.1316 - val_loss: 2.2594
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 2.0249 - val_loss: 2.0159
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.9571 - val_loss: 1.9456
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.8789 - val_loss: 1.9213
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.8233 - val_loss: 1.8924
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.7575 - val_loss: 1.8987



model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.6036 - val_loss: 3.7924
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.7765 - val_loss: 3.0022
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.4773 - val_loss: 2.5697
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.3218 - val_loss: 2.2606
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 2.2328 - val_loss: 2.0832
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 2.1748 - val_loss: 2.0248
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 2.1174 - val_loss: 1.9865
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 2.0617 - val_loss: 1.9640
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 2.0206 - val_loss: 1.9461
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.9758 - val_loss: 1.9334
Epoch 11/20
14725/14725 [==============================] - 15s - loss: 1.9546 - val_loss: 1.9148
Epoch 12/20
14725/14725 [==============================] - 15s - loss: 1.9045 - val_loss: 1.9121
Epoch 13/20
14725/14725 [==============================] - 15s - loss: 1.8757 - val_loss: 1.8888
Epoch 14/20
14725/14725 [==============================] - 15s - loss: 1.8437 - val_loss: 1.8874
Epoch 15/20
14725/14725 [==============================] - 15s - loss: 1.8145 - val_loss: 1.8822
Epoch 16/20
14725/14725 [==============================] - 15s - loss: 1.7805 - val_loss: 1.8785
Epoch 17/20
14725/14725 [==============================] - 15s - loss: 1.7558 - val_loss: 1.8868
Epoch 18/20
14725/14725 [==============================] - 15s - loss: 1.7218 - val_loss: 1.8670
Epoch 19/20
14725/14725 [==============================] - 15s - loss: 1.7032 - val_loss: 1.8759
Epoch 20/20
14725/14725 [==============================] - 15s - loss: 1.6832 - val_loss: 1.8834


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(GRU(256, return_sequences=True))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 33s - loss: 3.7814 - val_loss: 3.7282
Epoch 2/20
14725/14725 [==============================] - 29s - loss: 2.8453 - val_loss: 2.8542
Epoch 3/20
14725/14725 [==============================] - 29s - loss: 2.5178 - val_loss: 2.4434
Epoch 4/20
14725/14725 [==============================] - 29s - loss: 2.3762 - val_loss: 2.1894
Epoch 5/20
14725/14725 [==============================] - 29s - loss: 2.2896 - val_loss: 2.0862
Epoch 6/20
14725/14725 [==============================] - 29s - loss: 2.2254 - val_loss: 2.0516
Epoch 7/20
14725/14725 [==============================] - 29s - loss: 2.1565 - val_loss: 2.0133
Epoch 8/20
14725/14725 [==============================] - 29s - loss: 2.1132 - val_loss: 1.9992
Epoch 9/20
14725/14725 [==============================] - 29s - loss: 2.0798 - val_loss: 1.9881
Epoch 10/20
14725/14725 [==============================] - 29s - loss: 2.0509 - val_loss: 1.9784
Epoch 11/20
14725/14725 [==============================] - 29s - loss: 2.0198 - val_loss: 1.9618
Epoch 12/20
14725/14725 [==============================] - 29s - loss: 1.9822 - val_loss: 1.9383
Epoch 13/20
14725/14725 [==============================] - 29s - loss: 1.9437 - val_loss: 1.9300
Epoch 14/20
14725/14725 [==============================] - 29s - loss: 1.9198 - val_loss: 1.9163
Epoch 15/20
14725/14725 [==============================] - 29s - loss: 1.8989 - val_loss: 1.9160
Epoch 16/20
14725/14725 [==============================] - 29s - loss: 1.8866 - val_loss: 1.9085
Epoch 17/20
14725/14725 [==============================] - 29s - loss: 1.8493 - val_loss: 1.8965
Epoch 18/20
14725/14725 [==============================] - 29s - loss: 1.8248 - val_loss: 1.8878
Epoch 19/20
14725/14725 [==============================] - 29s - loss: 1.8037 - val_loss: 1.8870
Epoch 20/20
14725/14725 [==============================] - 29s - loss: 1.7724 - val_loss: 1.8862


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(GRU(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 32s - loss: 3.2809 - val_loss: 3.7595
Epoch 2/20
14725/14725 [==============================] - 29s - loss: 2.4379 - val_loss: 2.9869
Epoch 3/20
14725/14725 [==============================] - 29s - loss: 2.1504 - val_loss: 2.5361
Epoch 4/20
14725/14725 [==============================] - 29s - loss: 1.9887 - val_loss: 2.1294
Epoch 5/20
14725/14725 [==============================] - 29s - loss: 1.8984 - val_loss: 1.9727
Epoch 6/20
14725/14725 [==============================] - 29s - loss: 1.7892 - val_loss: 1.9264
Epoch 7/20
14725/14725 [==============================] - 29s - loss: 1.7172 - val_loss: 1.9100
Epoch 8/20
14725/14725 [==============================] - 29s - loss: 1.6361 - val_loss: 1.9124
Epoch 9/20
14725/14725 [==============================] - 29s - loss: 1.5621 - val_loss: 1.9122
Epoch 10/20
14725/14725 [==============================] - 29s - loss: 1.4863 - val_loss: 1.9045
Epoch 11/20
14725/14725 [==============================] - 29s - loss: 1.4150 - val_loss: 1.9278
Epoch 12/20
14725/14725 [==============================] - 29s - loss: 1.3691 - val_loss: 1.9181
Epoch 13/20
14725/14725 [==============================] - 29s - loss: 1.2970 - val_loss: 1.9414
Epoch 14/20
 1536/14725 [==>...........................] - ETA: 25s - loss: 1.1475
 
 
 
model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
 Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.1787 - val_loss: 3.9282
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.5070 - val_loss: 3.2479
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.2191 - val_loss: 2.7522
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.0637 - val_loss: 2.3331
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 1.9539 - val_loss: 2.0326
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.8622 - val_loss: 1.9440
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.7821 - val_loss: 1.9166
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.7169 - val_loss: 1.8996
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.6561 - val_loss: 1.8849
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.5910 - val_loss: 1.9032
Epoch 11/20
14725/14725 [==============================] - 15s - loss: 1.5082 - val_loss: 1.8878
Epoch 12/20
14725/14725 [==============================] - 15s - loss: 1.4513 - val_loss: 1.9252
Epoch 13/20
 7552/14725 [==============>...............] - ETA: 7s - loss: 1.3534
 
 
model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(BatchNormalization())
model.add(GRU(512))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
 Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 23s - loss: 3.1173 - val_loss: 3.8415
Epoch 2/20
14725/14725 [==============================] - 20s - loss: 2.4202 - val_loss: 3.1512
Epoch 3/20
14725/14725 [==============================] - 20s - loss: 2.1302 - val_loss: 2.7191
Epoch 4/20
14725/14725 [==============================] - 20s - loss: 1.9435 - val_loss: 2.3341
Epoch 5/20
14725/14725 [==============================] - 20s - loss: 1.7966 - val_loss: 1.9877
Epoch 6/20
14725/14725 [==============================] - 20s - loss: 1.6621 - val_loss: 1.9349
Epoch 7/20
14725/14725 [==============================] - 20s - loss: 1.5190 - val_loss: 1.9632
Epoch 8/20
14725/14725 [==============================] - 20s - loss: 1.3925 - val_loss: 1.9735
"""

NameError: name 'y' is not defined

In [ ]:
model.fit(X, y, epochs=5, batch_size=128, validation_split=0.1)

In [ ]:
test_author_model.summary()

In [22]:
generate(model, diversity=0.7, text="this is some test text does it really matter what it says " * 30)

----- Generating with seed: "oes it really matter what it says this is some test text does it really matter what it says this is "
oes it really matter what it says this is some test text does it really matter what it says this is 

TypeError: object of type 'numpy.float64' has no len()

In [24]:
def generate(model, diversity=0.5, text=""):
    """Generate text from a model"""
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(5000):
        x = np.zeros((1, maxlen), dtype=np.int)
        for t, char in enumerate(sentence):
            try:
                x[0, t] = char_indices[char]
            except:
                print(sentence)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    return

In [26]:
generate(model, diversity=0.5, text="this is some test text does it really matter what it says " * 30)

----- Generating with seed: " what it says this is some test text does it really matter what it says this is some test text does "
 what it says this is some test text does it really matter what it says this is some test text does the chan store if the store in for Vis a singic and the places I nor for the store in find store is a gitter beat to the trist and the counders here to find. The sele the store with the storit and the chansic I was next the back the truck out. This is a git and the courder the trust and the ching store with the me to good and the next the countret to the this store is little back to the clourder the selsection for the stration's and my for probebles to have the store if friends out befould the chan net store with the park is can sele, I was not longer casteral friends. The store store is little this is this lace in findst and the inclure the selmen store in finds a befter the beet the car this store if the chan strater and closed to the counders. The strail a

KeyboardInterrupt: 